# Урок 5. Сверточные нейронные сети для анализа текста.

ЗАДАНИЕ 1. 

Учим conv сеть для классификации - выбить auc выше 0.95

In [1]:
!pip install keras

In [18]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D,SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [3]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

import pandas as pd

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

print(df_train.head())
print(df_test.head())
print(df_val.head())


   id                                               text  class
0   0  @alisachachka не уезжаааааааай. :(❤ я тоже не ...      0
1   1  RT @GalyginVadim: Ребята и девчата!\nВсе в кин...      1
2   2  RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...      0
3   3  RT @epupybobv: Хочется котлету по-киевски. Зап...      1
4   4  @KarineKurganova @Yess__Boss босапопа есбоса н...      1
       id                                               text
0  204150   Тектоника и рельеф-самое ужасное в мире мучение(
1  204151  Ходили запускать шар желаний, но у нас не полу...
2  204152  Хочу лето только ради того, что бы направить н...
3  204153  RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4  204154  RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во...
       id                                               text  class
0  181467  RT @TukvaSociopat: Максимальный репост! ))) #є...      1
1  181468  чтоб у меня з.п. ежегодно индексировали на инд...      0
2  181469        @chilyandlime нехуя мне не хо

### Предобработка

In [4]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
tokens_filtered_top[:10]

['rt',
 'd',
 'хотеть',
 'знать',
 'ян',
 'мочь',
 'любить',
 'завтра',
 'мой',
 'хороший']

In [6]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [7]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [8]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [9]:
x_train.shape

(181467, 40)

In [11]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

y_train.shape

(181467, 2)

In [32]:
max_words = 5000
max_len = 300
num_classes = 2

# Training
epochs = 30
batch_size = 512
drop_embed = 0.2
dropout = 0.2
n_conv = 256
k_conv = 3
n_dim=64
n_dense=256

In [33]:
max_len

300

In [34]:
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [112]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=n_dim, input_length=max_len
                   ))
model.add(SpatialDropout1D(drop_embed))
model.add(Conv1D(n_conv, k_conv))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(n_dense))
model.add(Activation("relu"))
model.add(Dropout(dropout))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [113]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
#early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])#, early_stopping])

Epoch 1/30
319/319 [==============================] - 91s 284ms/step - loss: 0.5622 - accuracy: 0.6935 - val_loss: 0.5319 - val_accuracy: 0.7232
Epoch 2/30
319/319 [==============================] - 100s 312ms/step - loss: 0.5112 - accuracy: 0.7381 - val_loss: 0.5268 - val_accuracy: 0.7214
Epoch 3/30
319/319 [==============================] - 96s 302ms/step - loss: 0.4843 - accuracy: 0.7585 - val_loss: 0.5346 - val_accuracy: 0.7165
Epoch 4/30
319/319 [==============================] - 96s 300ms/step - loss: 0.4517 - accuracy: 0.7778 - val_loss: 0.5431 - val_accuracy: 0.7216
Epoch 5/30
319/319 [==============================] - 95s 297ms/step - loss: 0.4162 - accuracy: 0.7983 - val_loss: 0.5653 - val_accuracy: 0.7182
Epoch 6/30
319/319 [==============================] - 94s 296ms/step - loss: 0.3794 - accuracy: 0.8180 - val_loss: 0.5992 - val_accuracy: 0.7149
Epoch 7/30
319/319 [==============================] - 93s 292ms/step - loss: 0.3479 - accuracy: 0.8337 - val_loss: 0.6403 - val_a

In [38]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 67ms/step - loss: 1.0687 - accuracy: 0.7005


Test score: 1.0686968564987183
Test accuracy: 0.700524628162384


In [39]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 3s 73ms/step


**Вывод - на последней эпохи результат равен=accuracy: 0.92, что практически соответствует требумому.**

ЗАДАНИЕ 2. 

Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [42]:
from gensim.models import Word2Vec, FastText
import numpy as np
import pandas as pd

In [89]:
epochs = 10

In [70]:
modelW2V_train = Word2Vec(sentences=list(df_train["text"]), size=200, window=5, min_count=1, workers=32, seed=34)
x_test = Word2Vec(sentences=list(df_test["text"]), size=200, window=5, min_count=1, workers=32, seed=34)

In [71]:
modelW2V_train.train(list(df_train['text']),total_examples=len(list(df_train['text'])),epochs=20)

(57978139, 206345420)

In [92]:
modelW2V_train

In [97]:
def document_vector(doc):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in w2v.wv.vocab]
    
    return np.mean(w2v[doc], axis=0)

In [110]:
w2v=modelW2V_train
#df_train['xtrain_count']=df_train['text'].apply(lambda x: document_vector(x))
for row in np.arange(len(df_train)):
    try:
        df_train.at[row,'xtrain_count']= document_vector(df_train.at[row,'text'])
    except Exception:        
        pass

<ipython-input-97-a3de52c13398>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[doc], axis=0)


In [129]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
#early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit( np.asarray(list(df_train['xtrain_count']), dtype=np.int32), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])#, early_stopping])

ValueError: setting an array element with a sequence.

Не получается, непонятно как приводить Word2Vec к нужному типу для вставки как входной параметр в сеть